# Neuroscape playground

In [ ]:
from src.data import get_data
from src.utils import get_args_and_config
from src.train import train
from src.fmri import plot_brain

## data

In [ ]:
args, config = get_args_and_config()
data = get_data(args, config)

In [ ]:
params_lst = train(data, config)